In [12]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.rfr ,2.xgbr, 3.ann)

In [16]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in.h5')

#pickle for rfr
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
#    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_tikamaveragegcr5-5r6.h5')
# Show the model architecture
#new_model.summary()

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [19]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [20]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [21]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [22]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [23]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [24]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

ValueError: ignored

In [ ]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 4].argsort()]
    parents_pool=mixsort[:,0:4]
    parents_results=mixsort[:,4]
    return parents_pool

In [ ]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


#crossover_pool=crossover_process(parents_pool)
#print(crossover_pool)



In [ ]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [ ]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:4]
    return new_mating_pool


test2=survivor_selection()
print(test2)

In [ ]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


In [ ]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

In [ ]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))
